In [ ]:
!pip install --use-feature=2020-resolver https://s3-us-west-2.amazonaws.com/xgboost-nightly-builds/xgboost-1.3.0_SNAPSHOT%2B00b0ad1293b4fa74d6aca5da4e9ab7a9d16777f0-py3-none-manylinux2010_x86_64.whl

In [ ]:
import xgboost as xgb
xgb.__version__

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import zipfile
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import log_loss, roc_auc_score
import gc

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv')
sample_submission.tail()

In [ ]:
sample_submission.head()

In [ ]:
TRAIN_FOLDER = '../input/cats-and-dogs-embedded-data/train/train/'
TEST_FOLDER =  '../input/cats-and-dogs-embedded-data/test/test/'
IMG_SIZE = 224

In [ ]:
train_image_list = np.load('../input/cats-and-dogs-embedded-data/cats_and_dogs_1/train_image_list.npy')
test_image_list = np.load('../input/cats-and-dogs-embedded-data/cats_and_dogs_1/test_image_list.npy')

In [ ]:
train_image_list

In [ ]:
test_image_list

In [ ]:
ids = [int(x.split('.')[0]) for x in test_image_list]
ids[:20]

In [ ]:
target = np.load('../input/cats-and-dogs-embedded-data/cats_and_dogs_1/target.npy')

In [ ]:
target

In [ ]:
train_InceptionResNetV2 = np.load('../input/cats-and-dogs-embedded-data/cats_and_dogs_1/train_InceptionResNetV2.npy')
test_InceptionResNetV2 = np.load('../input/cats-and-dogs-embedded-data/cats_and_dogs_1/test_InceptionResNetV2.npy')

In [ ]:
train_InceptionResNetV2.shape

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_InceptionResNetV2, target, test_size=0.1, random_state=42)

In [ ]:
xgb_params = {
    'eta': 0.025,
    'max_depth': 7,
    'subsample': 0.8,
    'colsample_bytree': 0.7,
    'objective': 'reg:logistic',
    'eval_metric': 'auc',
    'verbosity': 0,
    'tree_method': 'hist', 
    'predictor': 'cpu_predictor'
}

dtrain = xgb.DMatrix(X_train, y_train)
dval = xgb.DMatrix(X_val, y_val)
dtest = xgb.DMatrix(test_InceptionResNetV2)

In [ ]:
del train_InceptionResNetV2, test_InceptionResNetV2, X_train, X_val
gc.collect()

In [ ]:
%%time
model = xgb.train(xgb_params, dtrain, num_boost_round=200)

In [ ]:
val_preds = model.predict(dval)

In [ ]:
val_preds

In [ ]:
y_val

In [ ]:
roc_auc_score(y_val, val_preds)

In [ ]:
log_loss(y_val, val_preds)

In [ ]:
%%time
val_shap_preds = model.predict(dval, pred_contribs=True)

In [ ]:
%%time
test_shap_preds = model.predict(dtest, pred_contribs=True)



In [ ]:
%%time
train_shap_preds = model.predict(dtrain, pred_contribs=True)

In [ ]:
test_preds = model.predict(dtest)

In [ ]:
test_preds.shape

In [ ]:
sample_submission['id'] = ids
sample_submission['label'] = test_preds
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head()